In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define paths
dataset_dir = 'tea sickness dataset'  # Replace with your dataset path
img_height, img_width = 150, 150  # Image dimensions
batch_size = 32
epochs = 20

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values to [0, 1]
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of data for validation
)

In [ ]:
# Load training data
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use for training
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use for validation
)

In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout to prevent overfitting
    Dense(train_generator.num_classes, activation='softmax')  # Output layer
])

In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('tea sickness dataset'):
    print(dirname)

In [ ]:
import tensorflow as tf

path='tea sickness dataset'

train,test = tf.keras.utils.image_dataset_from_directory(
    path,
    label_mode="categorical",
    validation_split=0.2,
    subset="both",
    seed=42,
    batch_size=32,
    image_size=(300, 300),
)

# Extract class names
class_names = train.class_names

train=train.cache().prefetch(1).shuffle(100)
test=test.cache().prefetch(1)

In [ ]:
from tensorflow.keras.applications import EfficientNetV2B3

base_model = EfficientNetV2B3(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
base_model.trainable = False

x = base_model.output
x = tf.keras.layers.Flatten()(x)  # Global average pooling to reduce spatial dimensions
x = tf.keras.layers.Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = tf.keras.layers.Dense(8, activation='softmax')(x)  # Output layer for 8 classes

# Create the final model
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

es=EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    verbose=1,
    mode='auto',
    restore_best_weights=True,
)
lr=ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=5,
    verbose=1,
    mode='auto',
    min_lr=1e-10,
)

In [ ]:
model.fit(
    train,
    epochs=10,
    validation_data=test,
    callbacks=[es,lr]
)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns


y_pred = model.predict(test)
y_pred_classes = np.argmax(y_pred, axis=1)


y_true = []
for images, labels in test:
    y_true.extend(np.argmax(labels.numpy(), axis=1))
y_true = np.array(y_true)


cm = confusion_matrix(y_true, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
# Save the model
model.save('tea_disease_efficientnet_model.h5')
print("Model saved as tea_disease_efficientnet_model.h5")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
loaded_model = load_model('tea_disease_efficientnet_model.h5')

# Function to preprocess the image
def preprocess_image(img_path, target_size=(300, 300)):
    img = image.load_img(img_path, target_size=target_size)  # Load and resize the image
    img_array = image.img_to_array(img)  # Convert to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    return img_array

# Function to predict the class of an image
def predict_image_class(img_path, model):
    preprocessed_img = preprocess_image(img_path)
    predictions = model.predict(preprocessed_img)
    predicted_class = np.argmax(predictions, axis=1)[0]  # Get the predicted class index
    confidence = np.max(predictions)  # Get the confidence score
    return predicted_class, confidence

# Test with a new image
new_image_path = 'IMG_20220503_143639.jpg'  # Replace with the path to your test image
predicted_class, confidence = predict_image_class(new_image_path, loaded_model)

# Map class index to class name
predicted_class_name = class_names[predicted_class]

print(f"Predicted Class: {predicted_class_name}")
print(f"Confidence: {confidence * 100:.2f}%")

In [ ]:
import numpy as np

# Save class_names to a file
np.save('class_names.npy', class_names)